In [1]:
#SETUP###############################################################
#Location to plot in map 1- Longitude 2- Latitude 3- Name of location
points = [(-87.94, 29.11, 'Marlin TLP'),
          (-87.5, 28.5, 'test')]
plot_point = True
#####################################################################
%matplotlib widget
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, AnchoredText, OffsetImage
plt.rcParams['font.sans-serif'] = "Segoe ui"
projection = ccrs.PlateCarree()

url1 = 'https://server.arcgisonline.com/arcgis/rest/services/Ocean/World_Ocean_Reference/MapServer/tile/{z}/{y}/{x}'
image1 = cimgt.GoogleTiles(url=url1, desired_tile_form="RGBA")
p = image1.crs
fig, ax = plt.subplots(figsize=(6.7, 4.7), dpi=110, subplot_kw={"projection": p})
dx = 325 / (111.12 * np.cos(np.deg2rad(points[0][1])))
ax.set_extent([points[0][0] - dx, points[0][0] + dx, points[0][1] - 1.6, points[0][1] + 1.6])

base_uri = 'https://server.arcgisonline.com/arcgis/rest/services/Ocean/World_Ocean_Base/MapServer/WMTS?'
base_uri2 = "https://server.arcgisonline.com/arcgis/rest/services/Ocean/World_Ocean_Reference/MapServer/WMTS?"
layer_name = 'Ocean_World_Ocean_Base'
layer_name2 = 'Ocean_World_Ocean_Reference'
ax.add_wmts(base_uri, layer_name=layer_name, )
ax.add_wmts(base_uri2, layer_name=layer_name2)

#Add map sources
SOURCE = 'Esri, GEBCO, NOAA, National Geographic, Garmin, HERE, Geonames.org, NGDC, and other contributors'
text = AnchoredText(SOURCE, loc=4, prop={'size': 6}, frameon=False)
ax.add_artist(text)

#Plot point and name
if plot_point:
    [plt.plot(i[0], i[1], transform=projection, marker='o', c='red', ms=6) for i in points]
    [ax.text(i[0] + .15, i[1] + .15, i[2], transform=projection, c='k',
            size=11, bbox=dict(boxstyle="round", ec='#121212', fc='#fadede', alpha=0.7)) for i in points]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
# Savefig
extent = list(ax.get_extent(crs=ccrs.PlateCarree()))
ax.set_extent(extent, crs=ccrs.PlateCarree())
xc, yc = p.transform_point(np.mean(extent[:2]), np.mean(extent[2:]), projection)
#add Fugro Logo
im_box = OffsetImage(plt.imread('Logo wordmark_RGB_EN_QB.png'), zoom=.015)
ab = AnnotationBbox(im_box, (xc, yc), (0.12, .87), 'data', "axes fraction", False, 0.1)
ax.add_artist(ab)
#Add compass image
im_box = OffsetImage(plt.imread('compass2.png'), zoom=.035)
ab = AnnotationBbox(im_box, (xc, yc), (0.94, .89), 'data', "axes fraction", False, 0.1)
ax.add_artist(ab)
#Make tmc horizontally centred on the middle of the map,
tmc = ccrs.TransverseMercator(np.mean(extent[:2]), np.mean(extent[2:]), approx=True)
#Get the extent of the plotted area in coordinates in metres
x0, x1, y0, y1 = ax.get_extent(tmc)
#Turn the specified scalebar location into coordinates in metres
sbx_le = x0 + (x1 - x0) * 0.05 #Generate the x coordinate for the ends of the scalebar
sbx_mi = sbx_le + 50000 #scale bar x position 50 kilometer
sbx_ri = sbx_le + 92600 #scale bar x position 50 nautical miles
sby_top = y0 + (y1 - y0) * 0.06  #scale bar y position top
sby_mid = y0 + (y1 - y0) * 0.05 #scale bar y position center
sby_bot = y0 + (y1 - y0) * 0.01  #scale bar y position bottom
#Plot the scalebar
ax.plot([sbx_le, sbx_mi], [sby_mid, sby_mid], transform=tmc, c='k', lw=1, marker=2, ms=7)
ax.plot([sbx_le, sbx_ri], [sby_mid, sby_mid], transform=tmc, c='k', lw=1, marker=3, ms=7)
#Plot the scalebar label
ax.text((sbx_le + sbx_mi)/2, sby_top, '50 km', transform=tmc, c='k', size=6, ha='center', va='bottom')
ax.text((sbx_le + sbx_ri)/2, sby_bot, '50 nm', transform=tmc, c='k', size=6, ha='center', va='bottom')
#add gridlines
gl = ax.gridlines(draw_labels=True, edgecolor='gray', linewidth=.75, linestyle=':')
gl.xlabel_style = {'size': 11,}
gl.ylabel_style = {'size': 11,}
#savefig
fig.savefig('proposal_frontispiece.png', dpi=110, bbox_inches='tight',)